In [1]:
import aoareader as reader
import torch
import time
import argparse

In [2]:
vocab_dict = torch.load('data/dict.pt')
train_data= torch.load('data/train.txt.pt')
valid_data=torch.load('data/dev.txt.pt')
train_dataset = reader.Dataset(train_data, 32, True)
valid_dataset = reader.Dataset(valid_data, 32, True, volatile=True)

In [3]:
print(train_data['documents'][0].size())
print(train_data['querys'][0].size())
print(train_data['answers'][0])
print(train_data['candidates'][0])
train_data.keys()

torch.Size([414])
torch.Size([24])
tensor(6669)
tensor([ 357,  641, 1085, 2887, 3024, 5467, 6361, 6669, 8641, 9207])


dict_keys(['documents', 'querys', 'answers', 'candidates'])

In [4]:
model = reader.AoAReader(vocab_dict, dropout_rate=0.1, embed_dim=384, hidden_dim=384)

/home/ttnguyen/anaconda3/envs/py36torch/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/ttnguyen/Projects/AoAreader/aoareader/AoAReader.py:56: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  weigth_init.orthogonal(weight.data)


In [5]:
model.cuda()

AoAReader(
  (embedding): Embedding(9631, 384, padding_idx=0)
  (gru): GRU(384, 384, batch_first=True, dropout=0.1, bidirectional=True)
)

In [6]:
(batch_docs, batch_docs_len, doc_mask), (batch_querys, batch_querys_len, query_mask), batch_answers, candidates = train_dataset[0]
print(batch_docs)
print(batch_querys)
print(batch_answers)
print(candidates)

tensor([[9473,  308, 2888,  ...,    0,    0,    0],
        [9473,  308, 2888,  ...,    0,    0,    0],
        [9473,  308, 2888,  ...,    0,    0,    0],
        ...,
        [5818,   16, 4416,  ...,    0,    0,    0],
        [5818,   16, 4416,  ...,    0,    0,    0],
        [  85, 9619, 3777,  ..., 1602,    2,    6]], device='cuda:0')
tensor([[6943, 8520, 9588,  ...,    0,    0,    0],
        [6943, 8520, 6669,  ...,    0,    0,    0],
        [6943, 8520, 6669,  ...,    0,    0,    0],
        ...,
        [ 391, 5108, 4168,  ..., 1602,    2,    6],
        [ 391, 5108, 4168,  ..., 1602,    2,    6],
        [8568, 9588,   12,  ...,    0,    0,    0]], device='cuda:0')
tensor([6669, 4715,  600, 4715, 1085, 3024, 6669, 3024, 3045, 4715, 3777, 5134,
        6669, 3024, 3024, 4715, 6669, 4715, 6669, 4715, 3024, 6669, 3024, 3024,
        4715, 4715, 5729,  641, 3024, 3026, 8552, 3026], device='cuda:0')
tensor([[ 357,  641, 1085, 2887, 3024, 5467, 6361, 6669, 8641, 9207],
        [ 

In [7]:
pred_answers, answer_probs = model(batch_docs, batch_docs_len, doc_mask, batch_querys, batch_querys_len, query_mask,answers=batch_answers, candidates=candidates)
print(pred_answers)
print(pred_answers.size())
print(answer_probs)
print(answer_probs.size())


tensor([6669, 4715, 6669, 4715, 6492, 6111, 6669, 6669, 6111, 4715, 4715, 4715,
        6669, 6669, 3024, 4715, 6669, 4715, 6669, 4715, 4715, 6669, 3024, 3024,
        3024, 6669, 4715, 3024, 3024, 6669, 5729, 8552], device='cuda:0')
torch.Size([32])
tensor([0.0098, 0.0121, 0.0024, 0.0142, 0.0024, 0.0048, 0.0078, 0.0027, 0.0059,
        0.0118, 0.0030, 0.0032, 0.0111, 0.0067, 0.0100, 0.0091, 0.0092, 0.0111,
        0.0083, 0.0089, 0.0090, 0.0115, 0.0113, 0.0111, 0.0110, 0.0110, 0.0026,
        0.0026, 0.0131, 0.0023, 0.0023, 0.0043],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
torch.Size([32])


In [8]:
def loss_func(answers, pred_answers, answer_probs):
    num_correct = (answers == pred_answers).sum().squeeze().data[0]
    loss = - torch.mean(torch.log(answer_probs),0, keepdim=True)
    return loss.cuda(), num_correct

loss, num_correct = loss_func(batch_answers, pred_answers, answer_probs)
print(loss)
print (num_correct)

tensor([5.0364], device='cuda:0', grad_fn=<NegBackward>)
tensor(16, device='cuda:0')


/home/ttnguyen/anaconda3/envs/py36torch/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  
